In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
import numpy as np
import scipy.signal
# render frames of single expanding circle
import imageio

#%matplotlib notebook
%matplotlib inline
mpl.rcParams['figure.figsize'] = (18, 6)


In [ ]:
import os, glob, sys

files = glob.glob('csv/c*.csv')
# make sure we get a consistent order.
files.sort()
data = []
count = 0
for name in files: 
    try:
        print " reading  file " + name
        thisfile = np.loadtxt(name, skiprows=1, usecols=tuple((0,1,2,3,4)),delimiter=",")
        thisfile = scipy.signal.decimate(thisfile, 10, n=None, ftype='iir', axis=0, zero_phase=True)
        data.append(thisfile)
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            pass
        else: # Do not fail if a directory is found, just ignore it.
            raise exc # Propagate other kinds of IOError.
    count += 1
    if count > 100:
        break
            
print "done"

In [ ]:
# massage data, correct for DST
day = 7420 # this many indexes in a day, roughly

hour = int(float(day)/24.) # chop this off to correct for dst
newd = []
d = data[0]
oldt = d[0,0]
dst = True
for i, d in enumerate(data[:]):
    if d.shape[0] > 7420:
        #print d.shape
        t = d[0,0]
        print(t -oldt)
        if (t - oldt) > 88000 and (t - oldt < 90000):
            dst = False
        if dst:
            # add in an hour to time 
            newd.append(d[hour:7420,:])
            print("DST")
        else:
            newd.append(d[0:7420-hour,:])
        oldt = t
            
        #del data
#print newd[0]
                   
    
np.save('cleandata.npy',newd)

In [2]:
# Make linear array of decimated data
import scipy

newd=np.load('cleandata.npy')

decd = []
numdays = 100
for i, d in enumerate(newd[0:100]):
    q = d[:,1:4]
    decd.append(scipy.signal.decimate(q, 10, axis=0, ftype='iir', zero_phase=True))

decd = np.array(decd) 

# start here to save memory
decd = np.load('decimated-days.npy')
ldays, lwidth, lcolors = decd.shape

print(decd.shape)

larr = decd.reshape(ldays*lwidth,lcolors)



(100, 712, 3)


In [4]:
# Make linear array of decimated data
import scipy



# scale to zero-1, add epsilon so root of zero does not happen
print(np.max(larr))
print(np.min(larr))
larr = larr - np.min(larr) + np.finfo(float).eps


#larr = larr/np.max(larr) 
larr = larr/np.max(larr) 



# gamma correct 
larr = (larr)**(1/1.8)

print(np.max(larr))
print(np.min(larr))

print "done"
print larr.shape

arrlen = ldays*lwidth


1.0
4.28589630395e-12
1.0
2.01176803353e-09
done
(71200, 3)


In [ ]:
# render frames of expanding circles
def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = 0
    if i < 0:
        i = 0
    return cmap[i,:]
    


width = 1920
height = 1080

#width = 300
#height = 200

out = np.zeros((height,width,3),dtype=float)

# this many frames in a day
framesperday = 240 # render at 30fps for smooth
# there are ldays of lwidth points in larray which is arrlen points long.
# we want lwidth/framesperday points to get n frames per day. Divide byarrlen to get
# scalefrac is the fraction (0-1) we should advance to get framesperday

scalefrac = float(lwidth)/(float(arrlen)*framesperday)


daysinframe = 2.5

# fraction of entire extent to use in one frame
# one day is lwidth points
ffrac = daysinframe*lwidth/float(arrlen)



offset = np.arange(0, 1.0 - ffrac, (1.0 - ffrac)*scalefrac)
#offset =[0.]
for i, off in enumerate(offset):
    if i > 15849:
        print(off)
        minc = 999
        maxc = -999
        for w in range(width):
            for h in range(height):
        
                x = (w - width/2)/float(width) 
                y = (h - height/2)/float(width)
                r = np.sqrt(x*x + y*y)
                #th = np.arctan2(y,x)
                #out[h,w] = getcolor(daysg[1200],0.5*(1 + off -r))
                out[h,w] = getcolor(larr,off + (1. -r)*ffrac)

                #if np.min(out[h,w]) < minc:
                #    minc = np.min(out[h,w]) 
                if np.max(out[h,w]) > maxc:
                    maxc = np.max(out[h,w]) 

        
        #print (minc, maxc)
        fname  = "long/f{:05d}.png".format(i)
        #fname  = "frames/f{:05d}.png".format(i)
        imageio.imwrite(fname, (255*out/maxc).astype(np.uint8))
        print "saved " + fname
        if i ==0 and False:
            fig = plt.figure(frameon=False, figsize = (12,12))
            plt.imshow(out)
            plt.axis('off')  
            fig.axes[0].get_xaxis().set_visible(False)
            fig.axes[0].get_yaxis().set_visible(False)
            #plt.savefig('advance{:03d}.png'.format(i),bbox_inches='tight', pad_inches=0)
            print "saved"
    
# to convert to gif with imagemagick
#$ /cygdrive/c/Program\ Files/ImageMagick-7.0.5-Q16/convert.exe   -delay 7 -loop 0 frame*.png ani.gif
#ffmpeg -r 15 -f image2  -i frame%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p  test.mp4

# imagemagic to get a gif
#!convert  -delay 7 -loop 0 frames/f*.png

#ffmpeg to get a mp4

#!ffmpeg -r 20 -f image2  -i long/f%05d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p -y longout.mp4


In [ ]:
# radial plot of grids for a particular day
import gc
gc.collect()

def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = l-1
    if i < 0:
        i = 0
    return cmap[i,:]
    


# total width and height
twidth = 1920
theight = 1080

#twidth=600
#theight=400

# with and height of grid, each grid element is a day
gwidth = 3
gheight = 2


gwidth=6
gheight=4

width = int(twidth/float(gwidth))
height = int(theight/float(gheight))



out = np.zeros((theight,twidth,3),dtype=float)



# this many frames in a day
framesperday = 240 # render at 30fps for smooth
# there are ldays of lwidth points in larray which is arrlen points long.
# we want lwidth/framesperday points to get n frames per day. Divide byarrlen to get
# scalefrac is the fraction (0-1) we should advance to get framesperday

scalefrac = float(lwidth)/(float(arrlen)*framesperday)


daysinframe = 2

# fraction of entire extent to use in one frame
# one day is lwidth points
ffrac = daysinframe*lwidth/float(arrlen)



offset = np.arange(0, 1.0 - ffrac, (1.0 - ffrac)*scalefrac)
#offset =[0.]

for i, off in enumerate(offset):

    # day count
    dc = 0
    print(off)
    minc = 999
    maxc = -999
    now = off
    for gw in range(gwidth):
        for gh in range(gheight):
    
            for w in range(width):
                for h in range(height):

                    x = (w - width/2)/float(width) 
                    y = (h - height/2)/float(width)
                    r = np.sqrt(x*x + y*y)
                    #th = np.arctan2(y,x)
                    #day = int(100*th*4/2*np.pi)
                    # want to show morning, so outside (max) is zero and inside (min) is 1/2
                    #print r
                    #out[(gh*height) + h, (gw*width) + w] = getcolor(daysg[dc,:,:],mapr(r))
                    #out[(gh*height) + h, (gw*width) + w] = getcolor(decd[dc,:,:],mapr(r))

                    color = getcolor(larr, now + (dc *0.55* ffrac) + (1. -r)*ffrac)

                    out[(gh*height) + h, (gw*width) + w] = color
                    #if np.min(out[h,w]) < minc:
                    #    minc = np.min(out[h,w]) 
                    if np.max(color) > maxc:
                        maxc = np.max(color) 


            print gw, gh, dc
            dc += 1
    fname  = "16x9/f{:05d}.png".format(i)
    #fname  = "frames/f{:05d}.png".format(i)
    imageio.imwrite(fname, (255*out/maxc).astype(np.uint8))
    print "saved " + fname

    # to convert to gif with imagemagick
    #$ /cygdrive/c/Program\ Files/ImageMagick-7.0.5-Q16/convert.exe   -delay 7 -loop 0 frame*.png ani.gif
#ffmpeg -r 15 -f image2  -i frame%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p  test.mp4


0.0
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00000.png
4.08333333333e-05
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00001.png
8.16666666667e-05
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00002.png
0.0001225
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00003.png
0.000163333333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00004.png
0.000204166666667
0 0 0
0 1 1
0 

0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00042.png
0.00175583333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00043.png
0.00179666666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00044.png
0.0018375
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00045.png
0.00187833333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00046.png
0.00191916666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 

1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00084.png
0.00347083333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00085.png
0.00351166666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00086.png
0.0035525
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00087.png
0.00359333333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00088.png
0.00363416666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 

3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00126.png
0.00518583333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00127.png
0.00522666666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00128.png
0.0052675
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00129.png
0.00530833333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00130.png
0.00534916666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3

4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00168.png
0.00690083333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00169.png
0.00694166666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00170.png
0.0069825
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00171.png
0.00702333333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f00172.png
0.00706416666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4

...

3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04196.png
0.1713775
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04197.png
0.171418333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04198.png
0.171459166667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04199.png
0.1715
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04200.png
0.171540833333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3

0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04239.png
0.173133333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04240.png
0.173174166667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04241.png
0.173215
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04242.png
0.173255833333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04243.png
0.173296666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1

3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04281.png
0.174848333333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04282.png
0.174889166667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04283.png
0.17493
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04284.png
0.174970833333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04285.png
0.175011666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 

0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04324.png
0.176604166667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04325.png
0.176645
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04326.png
0.176685833333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04327.png
0.176726666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04328.png
0.1767675
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5


3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04366.png
0.178319166667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04367.png
0.17836
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04368.png
0.178400833333
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04369.png
0.178441666667
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 3 19
5 0 20
5 1 21
5 2 22
5 3 23
saved 16x9/f04370.png
0.1784825
0 0 0
0 1 1
0 2 2
0 3 3
1 0 4
1 1 5
1 2 6
1 3 7
2 0 8
2 1 9
2 2 10
2 3 11
3 0 12
3 1 13
3 2 14
3 3 15
4 0 16
4 1 17
4 2 18
4 

In [ ]:
# radial plot of grids for a particular day
import gc
gc.collect()
def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = l-1
    if i < 0:
        i = 0
    return cmap[i,:]
    

def mapr(r):
    """map radius into a fraction of a colormap, 0 < r < .707"""
    return(2*r)

# width and height of inner frames
day = 20
froot = "day{}".format(day)
l, c = daysg[20].shape
width = 1000
height = 1000

# with and height of grid, each grid element is a day
gwidth = 1
gheight = 1

out = np.zeros((gheight*height,gwidth*width,3),dtype=float)
for gw in range(gwidth):
    for gh in range(gheight):
    
        for w in range(width):
            for h in range(height):

                x = (w - width/2)/float(width) 
                y = (h - height/2)/float(height)
                r = np.sqrt(x*x + y*y)
                #th = np.arctan2(y,x)
                #day = int(100*th*4/2*np.pi)
                # want to show morning, so outside (max) is zero and inside (min) is 1/2
                #print r
                out[(gh*height) + h, (gw*width) + w] = getcolor(daysg[day,:,:],mapr(r))
                #out[(gh*height) + h, (gw*width) + w] = getcolor(decd[dc,:,:],mapr(r))
        dc += 1
        print gw, gh, dc
    
fname  = froot + ".png"
imageio.imwrite(fname, (255*out).astype(np.uint8))
print "saved " + fname
fig = plt.figure(frameon=False, figsize = (12,12))
plt.imshow(out)
plt.axis('off')  
fig.axes[0].get_xaxis().set_visible(False)
fig.axes[0].get_yaxis().set_visible(False)
        #plt.savefig('advance{:03d}.png'.format(i),bbox_inches='tight', pad_inches=0)

write_hdr(out, froot + ".hdr")
print "saved"
    # to convert to gif with imagemagick
    #$ /cygdrive/c/Program\ Files/ImageMagick-7.0.5-Q16/convert.exe   -delay 7 -loop 0 frame*.png ani.gif
#ffmpeg -r 15 -f image2  -i frame%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p  test.mp4


In [ ]:
# radial plot

def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = l-1
    if i < 0:
        i = 0
    return cmap[i,:]
    

width = 400
height = 400
out = np.zeros((width,height,3),dtype=float)

offset = np.arange(-0.1,1.0, 0.002)
#offset =[0.]
for i, off in enumerate(offset):
    for w in range(width):
        for h in range(height):

            x = (w - width/2)/float(width) 
            y = (h - height/2)/float(height)
            r = np.sqrt(x*x + y*y)
            #th = np.arctan2(y,x)
            #day = int(100*th*4/2*np.pi)
            # want to show morning, so outside (max) is zero and inside (min) is 1/2
            out[h,w] = getcolor(daysg[1200],0.5*(1 + off -r))
        
        
    
    fname  = "frame{:03d}.png".format(i)
    imageio.imwrite(fname, (255*out).astype(np.uint8))
    print "saved " + fname
    if i ==0 :
        fig = plt.figure(frameon=False, figsize = (12,12))
        plt.imshow(out)
        plt.axis('off')  
        fig.axes[0].get_xaxis().set_visible(False)
        fig.axes[0].get_yaxis().set_visible(False)
        #plt.savefig('advance{:03d}.png'.format(i),bbox_inches='tight', pad_inches=0)
        print "saved"
    # to convert to gif with imagemagick
    #$ /cygdrive/c/Program\ Files/ImageMagick-7.0.5-Q16/convert.exe   -delay 7 -loop 0 frame*.png ani.gif
#ffmpeg -r 15 -f image2  -i frame%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p  test.mp4


In [ ]:
# radial plots

def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = l-1
    if i < 0:
        i = 0
    return cmap[i,:]
    

width = 1200
height = 1200
out = np.zeros((width,height,3),dtype=float)

for w in range(width):
    for h in range(height):

        x = (w - width/2)/float(width) 
        y = (h - height/2)/float(height)
        r = np.sqrt(x*x + y*y)
        th = np.arctan2(y,x)
        #print x, y, r
        #out[w,h,:] = [0, 0, 0]
        day = int(200*th*8/(2*np.pi))
        # want to show morning, so outside (max) is zero and inside (min) is 1/2
        out[h,w] = getcolor(daysg[day],0.5*(1-r))
        # want to show evening, so outside (max) is 1 and inside (min) is 1/2
        #out[h,w] = getcolor(days[200],1.2*(r + 1)/2)
        #out[h,w] = getcolor(days[200],r)
        
        
    

plt.figure(figsize = (12,12))
plt.imshow(out)
plt.axis('off')    
plt.savefig('radial4.png')

In [ ]:
# spiral plots
from PIL import Image, ImageDraw
im = Image.new('RGBA', (400, 400), (0, 255, 0, 0)) 
draw = ImageDraw.Draw(im) 
draw.line((100,200, 150,300), fill=128)
im.show()

def getcolor(cmap, frac):
    """ go fraction into cmap and return a color triple """
    l, w = cmap.shape
    assert(w ==3)
    i = int(l*frac)
    if i >= l:
        i = l-1
    if i < 0:
        i = 0
    return cmap[i,:]
    


In [ ]:
img.save('radialdays.png')

In [ ]:
def get_colors_for(c):
    red = c[:,1]
    grn = c[:,2]
    blu = c[:,3]

    mmax = np.max([np.max(red), np.max(blu), np.max(grn)])

    red = red/mmax
    grn = grn/mmax
    blu = blu/mmax


    colors = []
    for i, r in enumerate(red):
        colors.append(mpl.colors.to_rgb((red[i],grn[i],blu[i])))
    return colors

c = get_colors_for(newdata[0])


In [ ]:
newd = np.array(newd)
print newd.shape

In [ ]:
# make rectangular jpeg to wrap a sphere
import scipy.signal
from PIL import Image, ImageDraw

cmap = []
for d in newd[3:4]:
    print d.shape
    colors = d[:,1:4]
    # gamma correct
    colors = (colors / 65535.)**(1/2.2)
    colors = colors/np.max(colors)
    #colors = colors/mmax
    
    print colors.shape
    dcol =  scipy.signal.decimate(colors, 10, n=None, ftype='iir', axis=0, zero_phase=True)
    dcol_wide = np.multiply.outer(np.ones((1000)),dcol)
    cmap.extend(dcol_wide)
    
#dcol_wide = np.multiply.outer(np.ones((500)),dcol)
#dcol_wide = np.swapaxes(dcol_wide,1,2)
cmap = np.swapaxes(cmap,0,1)
cmap = np.array(cmap,dtype=float)
#print days.shape

#daysn = days/np.max(days)

#plt.imshow(cmap[1500:7500,:])
fig = plt.figure(frameon=False, figsize = (12,12))
plt.imshow(cmap[1500:7500,:])
plt.axis('off')  
fig.axes[0].get_xaxis().set_visible(False)
fig.axes[0].get_yaxis().set_visible(False)
plt.savefig('cmap.jpg',bbox_inches='tight', pad_inches=0)

In [ ]:
   fig = plt.figure(frameon=False, figsize = (12,12))
    plt.imshow(out)
    plt.axis('off')  
    fig.axes[0].get_xaxis().set_visible(False)
    fig.axes[0].get_yaxis().set_visible(False)
    plt.savefig('advance{:03d}.png'.format(i),bbox_inches='tight', pad_inches=0)